INGESTA PERU

QUERY ORIGEN

PREGUNTAS:

    1- Utilizamos CTEs o Tablas temporales para los calculos intermedios en SQL.
    2- A todo esto lo insertamos dentro de una celda Python en una variable Query o en una Tabla y de ahi lo llamamos en la celda de Python.

In [ ]:
WITH InitialSales AS (
    SELECT  
        a.cIDCompany AS Company_DwID,
        a.cIDInvoice AS Invoice_DwID,
        a.cSerie AS Serie,
        a.cForm AS Form,
        a.cIDCustomer AS Customer_DwID,
        a.cIDBranchInvoice AS BranchDwID,
        a.cIDTerritory AS Territory_DwID,
        a.dEmission AS Emission,
        a.xChannelDistribution AS ChannelDistribution,
        a.xInvoiceCancelReason AS InvoiceCancelReason,
        b.cIDOrder AS Order_DwID,
        a.xInvoiceType AS InvoiceType,
        e.cIDProduct AS Product_DwID,
        e.nAmount AS Amount,
        e.nUnitValue AS UnitValue,
        e.nTotalValue AS TotalValue,
        e.MC1LastUpdate,
        e.nFreegoods AS Freegoods,
        g.nSeq AS Seq,
        g.nValue AS [Value],
        g.cIDPricing AS Pricing_DwID,
        g.cIDStep AS Step_DwID,
        f.xIDReasonReturnStock AS ReasonReturnStock_DwID,
        h.lCalculated AS Calculated,
        h.cIDPricingFormationRule AS PricingFormationRule_DwID
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_Invoice a
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderInvoice b ON
        a.cIDCompany = b.cIDCompany AND
        a.cIDInvoice = b.cIDInvoice AND
        a.cForm = b.cForm AND
        a.cSerie = b.cSerie AND
        a.cIDBranchInvoice = b.cIDBranchInvoice
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItem e ON
        e.cIDCompany = a.cIDCompany AND
        e.cIDInvoice = a.cIDInvoice AND
        e.cForm = a.cForm AND
        e.cSerie = a.cSerie AND
        e.cIDBranchInvoice = a.cIDBranchInvoice
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItemDetail g ON
        g.cIDCompany = e.cIDCompany AND
        g.cIDInvoice = e.cIDInvoice AND
        g.cForm = e.cForm AND
        g.cSerie = e.cSerie AND
        g.cIDBranchInvoice = e.cIDBranchInvoice AND
        g.nSeq = e.nSeq AND
        g.cIDProduct = e.cIDProduct AND
        g.cIDStep IN ('00100', '00200', '00400', '00600', '00700', '00800', '00900', '01100', '01200', '01300', '00150')
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderExt f ON
        f.cIDOrder = b.cIDOrder AND
        f.cIDCompany = b.cIDCompany
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceRequestEmites m ON
        a.cIDCompany = m.cIDCompany AND
        a.cIDInvoice = m.cIDInvoice AND
        a.cForm = m.cForm AND
        a.cSerie = m.cSerie AND
        a.cIDBranchInvoice = m.cIDBranchInvoice AND
        UPPER(m.xInvoiceRequestStatus) = 'SUCCEEDED'
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_PricingFormationRule h ON
        g.cIDStep = h.cIDStep AND
        g.cIDPricing = h.cIDPricing AND
        g.cIDCompany = h.cIDCompany AND
        UPPER(h.cIDPricingFormationRule) != 'UNITVALUECALCULATED'
    WHERE
        a.mc1Enabled = 1 AND
        b.mc1Enabled = 1 AND
        e.mc1Enabled = 1 AND
        g.mc1Enabled = 1 AND
        f.mc1Enabled = 1 AND
        m.mc1Enabled = 1 AND
        h.mc1Enabled = 1 AND
        a.xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR') AND
        FORMAT(a.dEmission, 'yyyy-MM-dd') BETWEEN '2024-05-20' AND '2024-05-21'
),
ValidOrders AS (
    SELECT DISTINCT
        b.cIDOrder,
        b.cIDProduct,
        a.cIDBranchInvoice
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderInvoice a
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_OrderItem b ON
        a.cIDOrder = b.cIDOrder AND
        a.cIDCompany = b.cIDCompany AND
        a.cIDBranchInvoice = b.cIDBranchInvoice
    WHERE
        a.mc1Enabled = 1 AND
        b.mc1Enabled = 1 AND
        FORMAT(a.mc1LastUpdate, 'yyyy-MM-dd') BETWEEN '2024-05-20' AND '2024-05-21'
),
ValidSales AS (
    SELECT 
        a.*, 
        b.cIDOrder
    FROM InitialSales a
    INNER JOIN ValidOrders b ON
        a.Order_DwID = b.cIDOrder AND
        a.Product_DwID = b.cIDProduct AND
        a.BranchDwID = b.cIDBranchInvoice
),
Transformations AS (
    SELECT
        a.*,
        CASE WHEN a.Step_DwID = '00100' THEN [Value] ELSE 0 END AS Valor_Bruto,
        CASE WHEN a.Step_DwID = '00400' THEN [Value] ELSE 0 END AS Promocion_ML,
        CASE WHEN a.Step_DwID = '00600' THEN [Value] ELSE 0 END AS Descuento_ML,
        CASE WHEN a.Step_DwID = '00150' THEN [Value] ELSE 0 END AS Escalafc_ML,
        CASE 
            WHEN a.Step_DwID = '00200' AND a.Freegoods = 0 THEN a.[Value] 
            WHEN a.Step_DwID = '00100' AND a.Freegoods = 1 THEN a.[Value] 
            ELSE 0
        END AS Facdsl_ML,
        CASE WHEN a.Step_DwID = '01300' THEN [Value] ELSE 0 END AS Impuesto1,
        CASE WHEN a.Step_DwID = '01100' THEN [Value] ELSE 0 END AS Impuesto2,
        CASE WHEN a.Step_DwID = '01400' THEN [Value] ELSE 0 END AS Venta_Impuestos
    FROM ValidSales a
    WHERE a.cIDOrder IS NOT NULL
),
GroupedSales AS (
    SELECT
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue,
        SUM(Valor_Bruto) AS Valor_Bruto,
        SUM(Facdsl_ML) AS Facdsl_ML,
        SUM(Promocion_ML) AS Promocion_ML,
        SUM(Descuento_ML) AS Descuento_ML,
        SUM(Amount) AS Amount_Sum,
        SUM(Impuesto1) AS Impuesto1,
        SUM(Venta_Impuestos) AS Venta_Impuestos,
        SUM(Impuesto2) AS Impuesto2,
        SUM(Escalafc_ML) AS Escalafc_ML
    FROM Transformations
    GROUP BY
        Amount,
        BranchDwID,
        ChannelDistribution,
        Company_DwID,
        Customer_DwID,
        Emission,
        Form,
        Freegoods,
        Invoice_DwID,
        InvoiceCancelReason,
        InvoiceType,
        Order_DwID,
        Product_DwID,
        ReasonReturnStock_DwID,
        Seq,
        Serie,
        Territory_DwID,
        UnitValue
),
InvoiceItemDetailJoin AS (
    SELECT 
        a.cIDCompany AS Company_DwID,
        a.cIDBranchInvoice AS BranchInvoice_DwID,
        a.cSerie AS Serie,
        a.cIDInvoice AS Invoice_DwID,
        a.cForm AS Form,
        a.cIDProduct AS Product_DwID,
        FORMAT(a.MC1LastUpdate, 'yyyy-MM-dd hh:mm') AS MC1LastUpdate,
        c.nFreegoods AS FreeGoods,
        a.nValue AS nValue
    FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItemDetail a
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_Invoice b ON
        a.cIDBranchInvoice = b.cIDBranchInvoice AND
        a.cSerie = b.cSerie AND
        a.cIDInvoice = b.cIDInvoice AND
        a.cForm = b.cForm
    INNER JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_InvoiceItem c ON
        a.cIDBranchInvoice = c.cIDBranchInvoice AND
        a.cSerie = c.cSerie AND
        a.cIDInvoice = c.cIDInvoice AND
        a.cForm = c.cForm AND
        a.cIDProduct = c.cIDProduct AND
        a.nSeq = c.nSeq
    WHERE
        a.cIDStep = '00700' AND
        a.MC1Enabled = 1 AND
        b.MC1Enabled = 1 AND
        c.MC1Enabled = 1 AND
        FORMAT(b.dEmission, 'yyyy-MM-dd') BETWEEN '2024-05-20' AND '2024-05-21' AND
        b.xInvoiceType NOT IN ('WTM002', 'WTM005', 'ZOGRD', 'ZOGR')
),
InvoiceDetailGenerated AS (
    SELECT 
        Company_DwID,
        BranchInvoice_DwID,
        Serie,
        Invoice_DwID,
        Form,
        Product_DwID,
        MC1LastUpdate,
        FreeGoods,
        CASE WHEN FreeGoods = 1 THEN 0 ELSE nValue END AS nValue
    FROM InvoiceItemDetailJoin
),
InvoiceDetailGrouped AS (
    SELECT
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie,
        SUM(nValue) AS facdet
    FROM InvoiceDetailGenerated
    GROUP BY 
        Company_DwID,
        BranchInvoice_DwID,
        Form,
        Invoice_DwID,
        Product_DwID,
        Serie
),
FinalJoin AS (
    SELECT 
        a.*,
        b.facdet
    FROM GroupedSales a
    INNER JOIN InvoiceDetailGrouped b ON
        a.Company_DwID = b.Company_DwID AND
        a.BranchDwID = b.BranchInvoice_DwID AND
        a.Product_DwID = b.Product_DwID AND
        a.Serie = b.Serie AND
        a.Form = b.Form AND
        a.Invoice_DwID = b.Invoice_DwID
),
AddColumns AS (
    SELECT 
        a.Amount_Sum,
        a.BranchDwID,
        a.ChannelDistribution,
        a.Company_DwID,
        a.Customer_DwID,
        a.Descuento_ML,
        a.Escalafc_ML,
        a.Facdsl_ML,
        a.Freegoods,
        a.InvoiceCancelReason,
        a.Invoice_DwID,
        a.InvoiceType,
        a.Order_DwID,
        a.Promocion_ML,
        a.ReasonReturnStock_DwID,
        a.Seq,
        a.Serie,
        a.Territory_DwID,
        a.UnitValue,
        a.Valor_Bruto,
        a.Venta_Impuestos,
        a.Impuesto1,
        a.Impuesto2,
        CONCAT(a.Serie, '-', a.Invoice_DwID) AS Numero,
        a.ChannelDistribution AS Cod_Linea_ruta,
        a.Company_DwID AS Pais,
        a.InvoiceType AS AG19ID,
        a.Customer_DwID AS Codigo_Cliente,
        a.BranchDwID AS Codigo_Agencia,
        a.Territory_DwID AS Codigo_Ruta,
        a.Product_DwID AS Codigo_Producto,
        'FALSE' AS FACCON,
        CASE
            WHEN a.InvoiceType = 'ZBON' THEN 0 - facdet
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN 0 - facdet
            ELSE facdet
        END AS facdet,
        CASE 
            WHEN a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX' THEN a.ReasonReturnStock_DwID
            WHEN NOT (a.InvoiceType = 'ZREX' OR a.InvoiceType = 'ZOREX') THEN a.InvoiceCancelReason
            ELSE 'REVISAR'
        END AS Id_Motivo,
        FORMAT(CAST(a.Emission AS date), 'yyyy-MM-dd') AS Fecha,
        REPLACE(a.Territory_DwID, a.BranchDwID, '') AS Territory_DwID_Short
    FROM FinalJoin a
),
JoinWithSubchannel AS (
    SELECT *
    FROM (
        SELECT 
            a.*,
            b.cidbranchinvoice,
            ROW_NUMBER() OVER (PARTITION BY b.cidbranchinvoice, b.cidcustomer ORDER BY a.mc1Enabled DESC, a.mc1LastUpdate DESC) AS rn
        FROM CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_CustomerDivisionSectorExt a
        LEFT JOIN CBRTM3XPOLCD22_LASC1.GRO_BP_RTM.VW_MC1_CustomerDivisionSector b ON
            a.cidcustomer = b.cidcustomer
        WHERE 
            b.cidbranchinvoice IS NOT NULL
    ) sub
    WHERE rn = 1
),
FinalTablaSubchannel as 

SELECT * FROM JoinWithSubchannel;
